In [2]:
# import pyspark 
from pyspark import SparkContext
import pandas as pd

In [3]:
# creating a session 
session = SparkContext('local', 'Exam')


### Preprocessing to generate the RDD in the question

In [21]:

# read the data 
# data = pd.read_csv('final_exam_data.csv')
# data.head()
data = session.textFile('final_exam_data.csv')
data.collect()

# extracting the header 
header = data.first()

# getting the other rows 
data_rdd = data.filter(lambda row: row != header).map(lambda line: line.split(","))

data = data_rdd.collect()

# converting the last columns to int
for counter in range(len(data)):
    data[counter][0] = int(data[counter][0])
    data[counter][3] = int(data[counter][3])
    data[counter][4] = int(data[counter][4])
data_rdd = session.parallelize(data)

In [22]:
data_rdd.collect()
# now we can start solving our problem :) 

[[1, 'John', 'HR', 35, 50000],
 [2, 'Jane', 'IT', 28, 60000],
 [3, 'Bob', 'IT', 30, 75000],
 [4, 'Alice', 'Hr', 40, 55000],
 [5, 'Charlie', 'Finance', 45, 80000]]

## 1.Print the names of the employees that are older than 40


In [25]:
def get_older_than_40(item): 
    if item[3] > 40:
        return (item[1])

name_and_age_rdd = data_rdd.map(get_older_than_40)
print(name_and_age_rdd.collect())

[None, None, None, None, 'Charlie']


In [26]:
# this is a better solution, because we want to select some employees with certain tasks.
older_than_40 = data_rdd.filter(lambda row: row[3] > 40)
older_names = older_than_40.map(lambda row: row[1]).collect()
print("Employees older than 40:", older_names)


Employees older than 40: ['Charlie']


### 2. Print the sum of salaries in the IT department

In [30]:
# first we need to group them by the department
# 1. make a list of pairs, the key is the name, and the value is the age using the map
def get_departments_and_salary(item): 
    return (item[2], item[4])

dept_and_salaries = data_rdd.map(get_departments_and_salary)
print(dept_and_salaries.collect())

# then apply reduce by key 
salaries = dept_and_salaries.reduceByKey(lambda x,y: x + y)
print(salaries.collect())

# get the IT 
for item in salaries.collect():
    if item[0] == 'IT':
        print(item)


[('HR', 50000), ('IT', 60000), ('IT', 75000), ('Hr', 55000), ('Finance', 80000)]
[('HR', 50000), ('IT', 135000), ('Hr', 55000), ('Finance', 80000)]
('IT', 135000)


In [32]:
it_salaries = data_rdd.filter(lambda row: row[2] == "IT").map(lambda row: int(row[4]))
it_total_salary = it_salaries.reduce(lambda x, y: x + y) 
print("Total IT department salary:", it_total_salary)


Total IT department salary: 135000


In [36]:
it_sals = data_rdd.filter(lambda row: row[2] == 'IT')
it_sals.collect()

it_sals.map(lambda x: x[4]).collect()

[60000, 75000]

### Given a dataframe containing the employees data named EmpDF write PySpark code using DFs only to solve this:

In [37]:
# importing sparkSession
from pyspark.sql import SparkSession

# creating a session 
spark = SparkSession.builder.appName('Exam').getOrCreate()

# reading the csv in dataframe 
'''
    inferSchema to know the datatype automatically, otherwise it will be read as string.
    header to make the first row as the header, otherwise it will consider it as features.
'''
employees_df = spark.read.csv('final_exam_data.csv', inferSchema = True, header = True) 

In [39]:
employees_df.head(5)

[Row(Index=1, name='John', Department='HR', Age=35, Salary=50000),
 Row(Index=2, name='Jane', Department='IT', Age=28, Salary=60000),
 Row(Index=3, name='Bob', Department='IT', Age=30, Salary=75000),
 Row(Index=4, name='Alice', Department='Hr', Age=40, Salary=55000),
 Row(Index=5, name='Charlie', Department='Finance', Age=45, Salary=80000)]

In [ ]:
# now we can start solving the problems

### 1. Calculate the average salary of employees in each department

In [48]:
# do not forget to import the functions 
from pyspark.sql.functions import avg
# 1. we need to group the data by department first 
department_df = employees_df.groupBy('Department')
department_df.agg(avg("Salary")).show()

+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|        HR|    50000.0|
|   Finance|    80000.0|
|        Hr|    55000.0|
|        IT|    67500.0|
+----------+-----------+



In [49]:
avg_salary_df = employees_df.select("Department", "Salary").groupBy("Department").agg(avg("Salary"))
avg_salary_df.show()


+----------+-----------+
|Department|avg(Salary)|
+----------+-----------+
|        HR|    50000.0|
|   Finance|    80000.0|
|        Hr|    55000.0|
|        IT|    67500.0|
+----------+-----------+



### Print the name of the employee with the highest salary


In [55]:
# we need to select the max 
from pyspark.sql.functions import max
maxSalary = employees_df.select(max("Salary")).collect()[0][0]

In [56]:
maxSalary

80000

In [58]:
# now we can filter based on this salary 
names = employees_df.filter(employees_df['Salary'] == maxSalary)
name = names.collect()[0][1]
name 

'Charlie'